In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
from scipy.special import erfcinv as erfcinv
import tqdm as tqdm
import time
from models import gauss_func,gp_3D_solution,calculate_cost,gp_1D_solution,calculate_dcost
import matplotlib.pyplot as plt
from plotting import mole_fraction_plot
from read_settings import read_settings

In [ ]:
grid,source,atm,obs = read_settings('../site_settings.yml')
x = grid['advect_axis']
y = grid['crosswind_axis']
z = grid['vertical_axis']

# Inverse Problem: Estimate the Parameters from Data
## Perfect Model Experiment
### Generate some noisy observations

In [ ]:
obs_x = x[44]*np.ones(y.shape[0])
obs_y = y[:]
obs_z = np.zeros(y.shape[0])
observation_loc = {'x':obs_x,'y':obs_y,'z':obs_z}
obs_mole_fraction = gp_1D_solution(loc=observation_loc,atm=atm,source=source)+atm['background']
noisy_obs_mole_fraction = obs_mole_fraction + obs['is_noise']/100.*atm['background']*np.random.randn(*obs_mole_fraction.shape)
noisy_obs_mole_fraction_enh = noisy_obs_mole_fraction - atm['background']

observation = {'mole_fraction':noisy_obs_mole_fraction_enh,'uncert':obs['is_noise']/100.*atm['background']}
for ky in observation_loc.keys(): 
    observation[ky] = observation_loc[ky]

### Define the prior parameters

In [ ]:
prior_atm=atm.copy()
prior_source=source.copy()
prior_source['emis_rate'] = 0.5*source['emis_rate']
prior_source['emis_rate_uncert'] = 0.5*prior_source['emis_rate']

### Examine the Cost Function

In [ ]:
n_samples = 1000
emis_rate_ensemble = prior_source['emis_rate'] + prior_source['emis_rate_uncert']*np.random.randn(n_samples)
current_source=prior_source.copy()
current_atm=prior_atm.copy()
cost = np.zeros(n_samples)
dcost = np.zeros(n_samples)
loc = {'x':obs_x,'y':obs_y,'z':obs_z}

for i in range(n_samples):
    current_source['emis_rate'] = emis_rate_ensemble[i]
    emission = {'current':emis_rate_ensemble[i],'prior':prior_source['emis_rate'],'prior_uncert':prior_source['emis_rate_uncert']}
    cost[i] = calculate_cost(observation=observation,emission=emission,source=current_source,atm=current_atm)
    dcost[i] = calculate_dcost(observation=observation,emission=emission,source=current_source,atm=current_atm)
plt.scatter(emis_rate_ensemble,cost/cost.max(),s=1)
plt.scatter(emis_rate_ensemble,dcost/dcost.max(),s=1)
plt.legend(['Cost','Jacobian'])

### Gradient Descent Optimizer

In [ ]:
max_iterations = 1000
current_source = prior_source.copy()
observation = {'mole_fraction':noisy_obs_mole_fraction_enh,'uncert':obs['is_noise']/100.*atm['background']}
for ky in observation_loc.keys(): 
    observation[ky] = observation_loc[ky]
emission = {'current':prior_source['emis_rate'],'prior':prior_source['emis_rate'],'prior_uncert':prior_source['emis_rate_uncert']}
cost = np.zeros(max_iterations)
cost[0] = calculate_cost(observation=observation,emission=emission,source=current_source,atm=atm)
dcost = np.zeros(max_iterations)
dcost[0] = calculate_dcost(observation=observation,emission=emission,source=current_source,atm=atm)
emis = np.zeros(max_iterations)
emis[0]=prior_source['emis_rate']
iter=1
while (iter < max_iterations)*(np.abs(jac[iter-1]/jac[0])> 1e-2):    
    emis[iter] = emis[iter-1]-0.01*jac[iter-1]
    emission = {'current':emis[iter],'prior':prior_source['emis_rate'],'prior_uncert':prior_source['emis_rate_uncert']}
    current_source['emis_rate']=emis[iter]
    cost[iter] = calculate_cost(observation=observation,emission=emission,source=current_source,atm=atm)
    dcost[iter] = calculate_dcost(observation=observation,emission=emission,source=current_source,atm=atm)
    iter+=1
n_iter = iter-1
posterior={'emission':emission['current']}
posterior['mole_fraction']=gp_1D_solution(loc=observation_loc,atm=atm,source=current_source)
posterior['uncert']=((posterior['mole_fraction']**2/posterior['emission']**2).sum()/observation['uncert']**2 + 1./prior_source['emis_rate_uncert']**2)**(-1)

plt.plot(cost[:n_iter])
plt.title('Cost Function vs. Iterations');
plt.xlabel('Iteration Number');
plt.ylabel('Cost');

### Prior and Posterior Residuals

In [ ]:
prior_mole_fraction = gp_1D_solution(loc=observation_loc,atm=prior_atm,source=prior_source)
g1 = plt.hist(prior_mole_fraction-posterior['mole_fraction'],bins=np.linspace(-1,1,21));
g2 = plt.hist(posterior['mole_fraction']-observation['mole_fraction'],bins=np.linspace(-1,1,21),alpha=0.4);
plt.legend([g1[2],g2[2]],['Prior','Posterior'])
plt.xlabel('Model Data Mismatch (ppm)')